In [1]:
import pandas as pd
import numpy as np
import collections
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [2]:
testing = pd.read_csv('test.csv')
resources = pd.read_csv('cleaned_resources.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
testing.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects
0,p233245,5724a0c3ce11008366fff36dab4b943c,Ms.,CA,2016-04-27 13:45:41,Grades PreK-2,Music & The Arts,Visual Arts,Art Will Make You Happy!,My 2nd grade students are amazing! They are v...,My class is made up of 12 boys and 12 girls. ...,My second grade class will really benefit from...,The genorous donations to my project will make...,My students need a drying rack for their art p...,2
1,p096795,445619941dc7cbe81c7be109dc61a56a,Mrs.,SC,2016-04-28 12:43:56,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Keeping up with the TIMEs,Students within the classroom work in small gr...,My students are all very talented young indivi...,"We do a lot of small group, where the student ...",This project is very important to my classroom...,My students need Time Magazines for next year ...,1
2,p236235,e92a4902b1611a189643d6f12c51e6a0,Mrs.,SC,2016-04-29 21:16:05,Grades PreK-2,Math & Science,"Applied Sciences, Mathematics",Building Bridges to Problem Solving,My students share a love of learning. These s...,My class consists of 14 energetic learners. O...,These Fairy Tale Problem Solving STEM kits wil...,These materials will be help my students with ...,My students need to be mentally stimulated to ...,0
3,p233680,8e92622d2985d3faac1de71609c4be72,Mrs.,IA,2016-04-27 22:32:43,Grades PreK-2,Literacy & Language,Literacy,Classroom Library,Reading is the gateway to the soul. Guiding c...,First graders enter the classroom each day rea...,Book bins will help to organize our classroom ...,"When students begin the first grade, many are ...",My students need an organized classroom library.,0
4,p171879,91a3c89981f626d9a0d067c65fb186ce,Mr.,CA,2016-04-27 18:59:15,Grades 6-8,"Music & The Arts, Special Needs","Performing Arts, Special Needs",Reeds so we can Read,"\""Mr. Reyes! I need another reed!\"" I hear t...",We have a diverse population with almost entir...,Each day in my class students consume supplies...,My students come from very troubled homes and ...,My students need reeds to perform in class eac...,1


In [4]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78035 entries, 0 to 78034
Data columns (total 15 columns):
id                                              78035 non-null object
teacher_id                                      78035 non-null object
teacher_prefix                                  78034 non-null object
school_state                                    78035 non-null object
project_submitted_datetime                      78035 non-null object
project_grade_category                          78035 non-null object
project_subject_categories                      78035 non-null object
project_subject_subcategories                   78035 non-null object
project_title                                   78035 non-null object
project_essay_1                                 78035 non-null object
project_essay_2                                 78035 non-null object
project_essay_3                                 2704 non-null object
project_essay_4                                 2704

In [5]:
testing.shape

(78035, 15)

In [7]:
testing['project_submitted_datetime'] = pd.to_datetime(testing['project_submitted_datetime'])

In [8]:
testing = testing.merge(resources, how='left', left_on='id', right_on='id')

In [9]:
testing['project_subject_categories'] = testing['project_subject_categories'].str.replace('Warmth, Care & Hunger', 'Warmth Care & Hunger')
testing['project_subject_subcategories'] = testing['project_subject_subcategories'].str.replace('Warmth, Care & Hunger', 'Warmth Care & Hunger')

In [10]:
testing['project_subject_categories'] = list(testing['project_subject_categories'].str.split(r', '))
testing1 = testing['project_subject_categories'].apply(collections.Counter)
testing1 = pd.DataFrame.from_records(testing1).fillna(value=0)

In [11]:
testing['project_subject_subcategories'] = list(testing['project_subject_subcategories'].str.split(r', '))
testing2 = testing['project_subject_subcategories'].apply(collections.Counter)
testing2 = pd.DataFrame.from_records(testing2).fillna(value=0)

In [12]:
testing = testing.merge(testing1, how='left', left_index=True, right_index=True)
testing = testing.merge(testing2, how='left', left_index=True, right_index=True)

In [13]:
testing.drop(['project_subject_subcategories', 'project_subject_categories'], 1, inplace=True)

In [14]:
testing['project_title'] = testing['project_title'].str.lower()
testing['project_essay_1'] = testing['project_essay_1'].str.lower()
testing['project_essay_2'] = testing['project_essay_2'].str.lower()
testing['project_essay_3'] = testing['project_essay_3'].str.lower()
testing['project_essay_4'] = testing['project_essay_4'].str.lower()

In [16]:
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_title'] = testing['project_title'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)



testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)



testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)



testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)



testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)



testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])haven\'t(?![a-z])', 'have not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])i\'m(?![a-z])', 'i am', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])i\'ll(?![a-z])', 'i will', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])i\'ve(?![a-z])', 'i have', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])can\'t(?![a-z])', 'can not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])you\'re(?![a-z])', 'you are', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])isn\'t(?![a-z])', 'is not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])you\'ll(?![a-z])', 'you will', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])you\'ve(?![a-z])', 'you have', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])he\'s(?![a-z])', 'he is', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])he\'ll(?![a-z])', 'he will', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])she\'s(?![a-z])', 'she is', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])it\'s(?![a-z])', 'it is', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])it\'ll(?![a-z])', 'it will', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])we\'re(?![a-z])', 'we are', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])we\'ll(?![a-z])', 'we will', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])they\'re(?![a-z])', 'they are', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])that\'s(?![a-z])', 'that is', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])wasn\'t(?![a-z])', 'was not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])weren\'t(?![a-z])', 'were not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])aren\'t(?![a-z])', 'are not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])hasn\'t(?![a-z])', 'has not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])hadn\'t(?![a-z])', 'had not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])won\'t(?![a-z])', 'will not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])wouldn\'t(?![a-z])', 'would', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])don\'t(?![a-z])', 'do not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])doesn\'t(?![a-z])', 'does not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])didn\'t(?![a-z])', 'did not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])couldn\'t(?![a-z])', 'could not', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])shouldn\'t(?![a-z])', 'should not', case=False)

testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])inforgraphics(?![a-z])', 'information graphics', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])googlin(?![a-z])', 'googling', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])incoroporate(?![a-z])', 'incorporate', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])ultraportable(?![a-z])', 'ultra portable', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])succcesful(?![a-z])', 'successful', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])doble(?![a-z])', 'double', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])aromatherepy(?![a-z])', 'aromatherapy', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])resorces(?![a-z])', 'resources', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])strugglling(?![a-z])', 'struggling', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])consitstantly(?![a-z])', 'consistantly', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])piller(?![a-z])', 'pillar', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])technosavy(?![a-z])', 'tech-savy', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])unheathy(?![a-z])', 'unhealthy', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])experiance(?![a-z])', 'experience', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])somthing(?![a-z])', 'something', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])wonderoplois(?![a-z])', 'wonderopolis', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])visionmaker(?![a-z])', 'vision maker', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])visititing(?![a-z])', 'visiting', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])magnaformers(?![a-z])', 'magformers', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])scheving(?![a-z])', 'skeeving', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])juicin(?![a-z])', 'juicing', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])applocation(?![a-z])', 'application', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])fifeteen(?![a-z])', 'fifteen', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])dissappearing(?![a-z])', 'disappearing', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])lern(?![a-z])', 'learn', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])nesecarry(?![a-z])', 'necessary', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])instructuion(?![a-z])', 'instruction', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])famlies(?![a-z])', 'families', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])flipcamera(?![a-z])', 'flip camera', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])geaux(?![a-z])', 'go', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])fullly(?![a-z])', 'fully', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])fitbrain(?![a-z])', 'fit brain', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])gameup(?![a-z])', 'game up', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])typcially(?![a-z])', 'typically', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])helprobably(?![a-z])', 'he probably', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])chillout(?![a-z])', 'chill out', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])scientits(?![a-z])', 'scientists', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])subsistance(?![a-z])', 'subsistence', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])attaaack(?![a-z])', 'attack', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])healthysnacks(?![a-z])', 'healthy snacks', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])assessements(?![a-z])', 'assessments', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])allegeries(?![a-z])', 'allergies', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])dyanmath(?![a-z])', 'dynamath', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])spiltters(?![a-z])', 'splitters', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])surronded(?![a-z])', 'surrounded', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])utilitze(?![a-z])', 'utilize', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])nregular(?![a-z])', 'regular', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])readinglanugage(?![a-z])', 'reading language', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])acadamy(?![a-z])', 'academy', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])resortativr(?![a-z])', 'restorative', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])emphasising(?![a-z])', 'emphasizing', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])ctivity(?![a-z])', 'activity', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])hygeine(?![a-z])', 'hygiene', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])meterstick(?![a-z])', 'meter stick', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])expourse(?![a-z])', 'exposure', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])approachs(?![a-z])', 'approaches', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])andtwo(?![a-z])', 'and two', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])fitibits(?![a-z])', 'fit bits', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])thesee(?![a-z])', 'these', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])pesonally(?![a-z])', 'personally', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])efective(?![a-z])', 'effective', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])durring(?![a-z])', 'during', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])sdljlkjasdf(?![a-z])', '', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])indesdructable(?![a-z])', 'indestructable', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])responisble(?![a-z])', 'responsible', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])teammwork(?![a-z])', 'teamwork', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])mathmetician(?![a-z])', 'mathematician', case=False)
testing['project_resource_summary'] = testing['project_resource_summary'].str.replace(r'(?<![a-z])ofachild(?![a-z])', 'of a child', case=False)

In [ ]:
testing['project_title'] = testing['project_title'].str.replace(r'[^a-zA-Z ]+', '')
testing['project_essay_1'] = testing['project_essay_1'].str.replace(r'[^a-zA-Z ]+', '')
testing['project_essay_2'] = testing['project_essay_2'].str.replace(r'[^a-zA-Z ]+', '')
testing['project_essay_3'] = testing['project_essay_3'].str.replace(r'[^a-zA-Z ]+', '')
testing['project_essay_4'] = testing['project_essay_4'].str.replace(r'[^a-zA-Z ]+', '')
testing['project_essay_3'] = testing['project_essay_3'].fillna('')
testing['project_essay_4'] = testing['project_essay_4'].fillna('')
testing['essay1_count'] = testing['project_essay_1'].str.split().str.len()
testing['essay2_count'] = testing['project_essay_2'].str.split().str.len()
testing['essay3_count'] = testing['project_essay_3'].str.split().str.len()
testing['essay4_count'] = testing['project_essay_4'].str.split().str.len()
testing['proj_res_count'] = testing['project_resource_summary'].str.split().str.len()

In [ ]:
testing['project_title'] = testing['project_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
testing['project_essay_1'] = testing['project_essay_1'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
testing['project_essay_2'] = testing['project_essay_2'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
testing['project_essay_3'] = testing['project_essay_3'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
testing['project_essay_4'] = testing['project_essay_4'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
testing['project_resource_summary'] = testing['project_resource_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
testing['essay1_count_nostop'] = testing['project_essay_1'].str.split().str.len()
testing['essay2_count_nostop'] = testing['project_essay_2'].str.split().str.len()
testing['essay3_count_nostop'] = testing['project_essay_3'].str.split().str.len()
testing['essay4_count_nostop'] = testing['project_essay_4'].str.split().str.len()
testing['project_res_count_nostop'] = testing['project_resource_summary'].str.split().str.len()

In [ ]:
testing['month'] = testing['project_submitted_datetime'].dt.month
testing['year'] = testing['project_submitted_datetime'].dt.year

In [ ]:
testing.drop('teacher_id', 1, inplace=True)

In [ ]:
testing.drop('Warmth Care & Hunger_y', 1, inplace=True)

In [ ]:
testing.to_csv('test_clean.csv')